# Efficient LLM Training Tutorial

<a target="_blank" href="https://colab.research.google.com/github/MENA-ML/tutorials2025-tasks/blob/main/efficient_llm_training/LLM_Finetuning_Task.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This is the tutorial for the **2025 Middle East and North Africa Machine Learning (MenaML) Winter School**!

This tutorial will explore the fundamental aspects of Finetuning Large Language Models for specific tasks or domains. Basic Python programming skills are expected. Prior knowledge of standard LLM components (e.g. transformers, autoregression) is beneficial but optional when working through the notebooks as they assume minimal prior knowledge.

This tutorial combines detailed analysis and development of essential LLM Finetuning concepts via a practical exercise. Other necessary components will be developed using PyTorch. As a result, the tutorial offers deep understanding and facilitates easy usage in future applications.

# Notation
Sections marked with [📚] contain cells that you should read, modify and complete to understand how your changes alter the obtained results.
External resources are mentioned with [✨]. These provide valuable supplementary information for this tutorial and offer opportunities for further in-depth exploration of the topics covered.

# Libraries
This tutorial leverages PyTorch for transformer implementation and training, complemented by standard Python libraries for data processing and the Hugging Face datasets and transformer library.


# Hardware

GPU access is recommended for optimal performance, particularly for model training and text generation. While all code can run on CPU, a CUDA-enabled environment will significantly speed up these processes.

To grab a GPU (if available), make sure you go to Edit -> Notebook settings and choose a GPU under Hardware accelerator. Prefer **A100s** for higher flops.

# Credits
The tutorial is created by:

[Pranali Yawalkar](https://www.linkedin.com/in/pranali-yawalkar/)

It is inspired by and synthesizes various online resources, which are cited throughout for reference and further reading.

# Prerequisites

Verified account on http://huggingface.co

# Step 1. Hugging Face Auth

To complete this tutorial, we need access to Gemma models on huggingface.

1. You first need to follow the setup instructions at [HuggingFace User Tokens](https://huggingface.co/docs/hub/en/security-tokens) to create a new user access token.

  * Choose **finegrained** scope of the token
  * Ensure that **Read access to contents of all public gated repos you can access** is enabled under Profile -> Edit Access Token Permissions
![](https://drive.google.com/uc?export=view&id=18JamIbfBMfd9bJ3uuetso1l12nmEsB4C)

  * Copy and paste the token to colab secrets with key **HF_TOKEN**

2. Get access to Google Gemma 2B model on [huggingface.co](https://huggingface.co/google/gemma-2b).
3. When running the next cell, you'll be prompted with "Grant access?" messages - agree to provide colab secret access.

In [ ]:
import os
from google.colab import userdata

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
os.environ["WANDB_DISABLED"] = "true"

# Step 2. Install Dependencies

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1

# 📚 Step 3. Load Gemma Model

## [Optional] Model Architecture

For this tutorial, we'll be working with **Google Gemma 2B pretrained** model.



1.   Decoder only model
2.   Trained on context length of 8192 tokens ~ 6144 words (using the rule of thumb of 100 tokens ~= 75 words) at a time
3.   More details on the non linear activation function, differences with the 7B model and explanations of various model dimensions at [Google Developers Site](https://developers.googleblog.com/en/gemma-explained-overview-gemma-model-family-architectures/)
4.   We trade-off quality for lower computational cost and time, by choosing a 2B model for this exercise.

![](https://drive.google.com/uc?export=view&id=1B2L0-J9qJRuCLWcso3yIbR9-dpjbvj7Q)   


In [ ]:
# @title 📚  Load Model

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GemmaTokenizer, GenerationConfig


MODEL_ID = 'google/gemma-2b'

def reload_model():
  if 'MODEL' in globals():
    del globals()['MODEL']

  gc.collect()
  torch.cuda.empty_cache()

  model = AutoModelForCausalLM.from_pretrained(
      MODEL_ID,
      quantization_config=None,
      device_map={'': 0},
      token=os.environ['HF_TOKEN'],
  )
  return model


global TOKENIZER, MODEL
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_ID,
                                          token=os.environ['HF_TOKEN'])
MODEL = reload_model()


def query_model(query: str) -> str:
  # EXERCISE 1. Read PyTorch based documentation at
  # ✨ https://huggingface.co/docs/transformers/en/main_classes/text_generation
  # and complete this function
  ...

In [ ]:
# @title Sample Query

def sample_query(
    query: str = (
        "Janet's ducks lay 16 eggs per day. She eats three for "
        "breakfast every morning and bakes muffins for her friends every day "
        "with four. She sells the remainder at the farmers' market daily for "
        "$2 per fresh duck egg. How much in dollars does she make every day at "
        "the farmers' market?")):
  print('Query: ', query)
  print('Response: ', query_model(query))

sample_query()

## Trainable Params

These parameters are learnable parameters and are updated during training such that the predictive power of the model increases.


1.   **Weights**: Measure the importance of each input or feature in predicting

2.   **Biases**: A constant term added to the sum of weighted inputs
the output



In [ ]:
def print_trainable_parameters(model):
  """Prints the number of trainable parameters in the model."""
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} ||"
      f" trainable: {100 * trainable_params / all_param:.2f}%"
  )

In [ ]:
print_trainable_parameters(MODEL)

# Step 4. LLM Training Fundamentals



## Finetuning Introduction

In the world of large language models (LLMs), finetuning takes a pre-trained model (already trained on a massive amount of text data) and adapts it to perform better on a specific task, like:

* Translating languages
* Writing different kinds of creative content
* Answering your questions in a more informative way

Essentially, you're taking a general-purpose LLM and making it an expert in a particular area.

## Finetuning vs Pretraining key differences:

# TODO(pranalipy) -- probe the audience here before showing the text

![](https://drive.google.com/uc?export=view&id=1u_ha15E0oNwmb1BPFLNtySls9qxBaJU7)   


## Finetuning Training Workflow

A typical training workflow looks like:

1. Identify the task you want to make the LLM better at 🧑
2. Create evaluation data and framework that we want the LLM to get better at ❗
3. Pick/ create datasets aka ground truth that truly represent the task 📚. Run:

        *   Optional padding
        *   Optional truncation
        *   Shuffling
        *   Batching
        *   Repeating

4. Training loop with various hyperparameters ➰:
        *   Num epochs
        *   Batch size
        *   Max steps
        *   Learning Rate
        *   Optimizer -- rarely changed
        *   Warm up steps
5. Validation set for picking the best checkpoint ✅
6. Rerun evaluations ❗
7. Save the new checkpoint 💾


# 📚 Step 5. Load GSM8k data for Training and Evals

In [ ]:
from datasets import Dataset
from datasets import load_dataset

# EXERCISE 2: load the `gsm8k` dataset of `main` revision using `load_dataset`
# ✨ Documentation at https://huggingface.co/docs/datasets/en/loading

gsm8k = ...

# EXERCISE 3: Get the `train` and `test` splits of this dataset and print
# their lengths

gsm8k_train, gsm8k_test = ...

In [ ]:
def cleanup_answer(example):
  splits = example['answer'].split('####')
  example['answer'] = splits[0].strip() + '\nThe answer is ' + splits[1].strip()
  return example

print('Lenghts before cleanup, ', len(gsm8k_train), len(gsm8k_test))

# EXERCISE 4: Map all entries of gsm8k_train and gsm8k_test with the
# cleanup_answer function
gsm8k_train = ...
gsm8k_test = ...

print('Lenghts after cleanup, ', len(gsm8k_train), len(gsm8k_test))

# EXERCISE 5: Sample 10 entries from gsm8k_test to create a small eval set
small_gsm8k_test = ...

In [ ]:
def formatting_func(dataset: Dataset) -> list[str]:
  ###
  # EXERCISE 6: Format entries of example to follow a pattern like
  #
  # "Question: ..."
  # "Answer: ..."
  # EOS
  #
  # ✨ Documentation: https://huggingface.co/docs/trl/en/sft_trainer
  # ✨ EOS documentation: https://www.natebrake.com/blog/llm/end-of-sequence-explained
  # As a fun exercise, try finetuning without EOS!
  ###
  ...

In [ ]:
print(gsm8k_train[0])
print(gsm8k_train[1])

# Step 6. Prepare Evals

In [ ]:
# @title Sample Eval
sample_query()

In [ ]:
# @title Bulk Evals


# Sometimes the model outputs may not adhere to the provided format.
def run_bulk_evals(
    eval_set: Dataset,
    prompt_prefix: str = '',
    prompt_suffix: str = '',
    answer_pattern: str = 'The answer is',
    verbose: bool = False,
):
  correct = 0
  for i in range(len(eval_set)):
    problem = eval_set[i]
    eval_prompt = prompt_prefix + problem['question'] + prompt_suffix
    response = query_model(eval_prompt)

    expected_response = problem['answer']
    if answer_pattern in response:
      lines = response.splitlines()
      for l in reversed(lines):
        if answer_pattern in l:
          actual_response = l.split(answer_pattern)[1].strip()
          actual_response = (
              actual_response[:-1]
              if actual_response.endswith('.')
              else actual_response
          )
          parsed_expected_response = expected_response.split(answer_pattern)[
              1
          ].strip()
          print(
              'Parsed responses: ',
              actual_response,
              ' and ',
              parsed_expected_response,
          )
          if actual_response == parsed_expected_response:
            correct += 1
          break

    if verbose:
      print(
          '\nExpected response: \n',
          expected_response,
          ' \n\nFound response: ',
          response,
          '\n********************************\n',
      )
  return correct / len(eval_set)

In [ ]:
accuracy = run_bulk_evals(
    small_gsm8k_test,
    prompt_prefix='Answer the following question.\n',
    prompt_suffix=(
        '\nYou **must** end your answer with a line "The answer is " which'
        ' contains the numerical answer such that it is easy for me to parse'
        ' the answer. \n'
    ),
    verbose=True
)
print('Accuracy: ', accuracy)

# 📚 Step 6. Prepare Train Utils

In [ ]:
from peft import LoraConfig
import transformers
from transformers import Trainer, TrainingArguments
from trl import SFTTrainer


def train(
    *,
    model: transformers.PreTrainedModel = MODEL,
    num_epochs: int = 50,   # Note: Why did we set it so high?
    learning_rate: float = 2e-5,
    dataset: Dataset = gsm8k_train,
    lora_config: LoraConfig = None,
):
  # EXERCISE 6: Create TrainingArguments by reading
  # ✨ https://huggingface.co/docs/transformers/v4.48.0/en/main_classes/trainer#transformers.TrainingArguments
  # with the above epochs, learning rate, ✨`paged_adamw_8bit` optimizer
  # and bf16 mode.

  # EXERCISE 7: What's the best strategy to set the batch size?
  args = transformers.TrainingArguments()

  # Updates model in place
  trainer = SFTTrainer(
      model=model,
      train_dataset=dataset,
      args=args,
      peft_config=lora_config,
      formatting_func=formatting_func,
  )
  trainer.train()
  return trainer

# 📚 Step 8. Finetune!

## Libraries

✨ https://huggingface.co/docs/trl/en/sft_trainer

## 📚  Train

In [ ]:
MODEL = reload_model()
trainer = train(model=MODEL)

# EXERCISE 8: Identify what batch size was used for training

In [ ]:
# EXERCISE 9: explore learning rates in the set
# [5e-2, 5e-3, 2e-4, 5e-4, 2e-5, 5e-5] to find the best
# one for our setup

# Usually we use validation set loss for identifying the best LR.
# For simplicity, we'll use training loss as a proxy.

## Eval

In [ ]:
# @title Sample Eval Again

query_model(
    'Question: Janet’s ducks lay 16 eggs per day. She eats three for breakfast'
    ' every morning and bakes muffins for her friends every day with four. She'
    " sells the remainder at the farmers' market daily for $2 per fresh duck"
    " egg. How much in dollars does she make every day at the farmers' market?"
    ' \nAnswer: '
)

In [ ]:
# @title Bulk Evals Again
accuracy = run_bulk_evals(
    small_gsm8k_test,
    prompt_prefix='Question: ',
    prompt_suffix=' \nAnswer: ',
    answer_pattern='The answer is',
    verbose=True
)
print('Accuracy: ', accuracy * 100, '%')

# 📚  Step 9. LoRA Finetune!

## LoRA Concepts

![](https://drive.google.com/uc?export=view&id=1534a4LeyfTSegnm4Dnfxx6_mWB0BMUOZ)


1. **Efficient Parameter Adjustment**: Instead of modifying all the original model's parameters (which are numerous and resource-intensive), LoRA introduces trainable rank decomposition matrices into each layer of the Transformer architecture. These matrices have a much smaller size than the original model's weight matrices.

![](https://drive.google.com/uc?export=view&id=1Kz68zGJ_9IuZWQ-0tcGaD-Azm5G4GDCW)


2. **Reduced Memory and Comppute Footprint**: By training only these smaller matrices, LoRA significantly reduces the memory requirements and computational costs associated with fine-tuning large models.


3. **Avoids Catastrophic Forgetting**: The original model's weights remain frozen, and the LoRA matrices capture the task-specific knowledge, allowing the model to adapt to new data or tasks without altering its core capabilities.


4. **Merging with the Original Model**: During inference, the low-rank matrices are merged back with the original weights, effectively injecting the learned knowledge into the model.


5. **Modularity and Reusability**: The trained LoRA modules can be easily shared, reused, and combined with different pre-trained models, enabling flexible and efficient adaptation across various tasks.


**LoRA has been shown to achieve comparable or even better performance than full fine-tuning, while using significantly fewer resources.**


## Libraries

*   ✨ https://huggingface.co/docs/trl/en/sft_trainer
*   ✨ https://huggingface.co/docs/peft/en/package_reference/lora

## LoRA Hypers

✨ [LoRA Hypers doc](https://medium.com/@fartypantsham/what-rank-r-and-alpha-to-use-in-lora-in-llm-1b4f025fd133)

![](https://drive.google.com/uc?export=view&id=1eGxjTrnkpHxM8VN_SKJLa6yff0gmxJYP)



1.   **r**: rank of the LoRA matrices that capture information about our task. Higher the better. Typically, **low ranks: 8, 16, 32** and **high ranks: 128, 256**
2.   **lora_alpha**: Alpha is a scaling factor -- it changes how the LoRA's weights are weighed against the base model's. Higher alpha means the LoRA layers act more strongly than the base model. Typically set to **r** or **2 x r**. The original LoRA method uses the scalar function **lora_alpha/r** for scaling LoRA weights during forward pass.


##  📚 Train

In [ ]:
from peft import LoraConfig

base_lora_config = LoraConfig(
    r=64,
    lora_alpha=64,
    lora_dropout=0.01,
    bias="none",
    target_modules=[
        "q_proj",
        "o_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    task_type="CAUSAL_LM",
)

In [ ]:
# EXERCISE 10: Hyper r from [32, 64, 128] and lora_alpha from [r, 2*r]
MODEL = reload_model()
train(model=MODEL, lora_config=lora_config)
print(print_trainable_parameters(MODEL))

## Eval

In [ ]:
# @title Sample Eval Again
query_model(
    'Question: Janet’s ducks lay 16 eggs per day. She eats three for breakfast'
    ' every morning and bakes muffins for her friends every day with four. She'
    " sells the remainder at the farmers' market daily for $2 per fresh duck"
    " egg. How much in dollars does she make every day at the farmers' market?"
    ' \nAnswer: '
)

In [ ]:
# @title Bulk Evals Again
accuracy = run_bulk_evals(
    small_gsm8k_test,
    prompt_prefix='Question: ',
    prompt_suffix=' \nAnswer: ',
    answer_pattern='The answer is',
    verbose=True,
)
print('Accuracy: ', accuracy * 100, '%')

# 📚  [Advanced] Step 10. Custom PEFT

## Libraries



1.   [SFTTrainer](https://github.com/huggingface/trl/blob/main/trl/trainer/sft_trainer.py)
2.   [PEFT](https://github.com/huggingface/peft)
3.   [Transformers](https://github.com/huggingface/transformers)
4. [PyTorch NN](https://pytorch.org/docs/stable/nn.html)



## 📚  Create Custom LoRA

In [ ]:
from peft.tuners.lora.layer import Linear
import torch.nn as nn


class CustomLinearLoRA(Linear):

  def update_layer(
      self,
      adapter_name,
      r,
      lora_alpha,
      lora_dropout,
      init_lora_weights,
      use_rslora=False,
      use_dora=False,
  ):
    # This code works for linear layers, override for other layer types
    if r <= 0:
      raise ValueError(
          f"`r` should be a positive integer value but the value passed is {r}"
      )

    self.r[adapter_name] = r
    self.lora_alpha[adapter_name] = lora_alpha

    # EXERCISE 11: Define a dropout layer
    lora_dropout_layer = ...

    self.lora_dropout.update(nn.ModuleDict({adapter_name: lora_dropout_layer}))

    # Actual trainable parameters
    # EXERCISE 12: write a linear layer that goes from self.in_features to r
    self.lora_A[adapter_name] = ...
    # EXERCISE 13: write a linear layer that goes from r to self.out_features
    self.lora_B[adapter_name] = ...

    self.scaling[adapter_name] = lora_alpha / r

    self.reset_lora_parameters(adapter_name, init_lora_weights)
    self.set_adapter(self.active_adapters)

  def forward(self, x, *args, **kwargs):
    result = self.base_layer(x, *args, **kwargs)
    torch_result_dtype = result.dtype
    for active_adapter in self.active_adapters:
      if active_adapter not in self.lora_A.keys():
        continue
      lora_A = self.lora_A[active_adapter]
      lora_B = self.lora_B[active_adapter]
      dropout = self.lora_dropout[active_adapter]
      scaling = self.scaling[active_adapter]

      x = x.to(lora_A.weight.dtype)

      x = dropout(x)

      # EXERCISE 14: add to the result of the base layer, the output of
      # lora_B and lora_A and multiply by the scaling factor
      result = ...

    result = result.to(torch_result_dtype)

    return result

## Orchestrating our PeFT Model with Huggingface

Since we are using the HuggingFace PEFT library framework, we need to tweak some of its internal workings to be able to expose the LoRA layer above. Therefore the following cell is not very insightful to understand during this tutorial.

In [ ]:
from peft import LoraConfig, LoraModel, PeftModel, get_peft_model
from peft.tuners.lora.layer import Linear, dispatch_default
from peft.tuners.tuners_utils import BaseTunerLayer
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, AutoTokenizer
from transformers.utils import PushToHubMixin


def custom_dispatch_default(
    target: torch.nn.Module, adapter_name, lora_config, **kwargs
):
  new_module = None
  target_base_layer = (
      target.get_base_layer() if isinstance(target, BaseTunerLayer) else target
  )

  if isinstance(target_base_layer, torch.nn.Linear):
    kwargs.update(lora_config.loftq_config)
    new_module = CustomLinearLoRA(target, adapter_name, **kwargs)

  if new_module is None:
    new_module = dispatch_default(
        target, adapter_name, lora_config=lora_config, **kwargs
    )
  return new_module


class CustomLoraModel(LoraModel):

  @staticmethod
  def _create_new_module(lora_config, adapter_name, target, **kwargs):
    return custom_dispatch_default(
        target, adapter_name, lora_config=lora_config, **kwargs
    )


class CustomPeftModel(PeftModel):

  def __init__(self, model, peft_config, adapter_name="default"):
    PushToHubMixin.__init__(self)
    torch.nn.Module.__init__(self)

    self.modules_to_save = None
    self.active_adapter = adapter_name
    self.peft_type = peft_config.peft_type
    # These args are special PEFT arguments that users can pass.
    # They need to be removed before passing them to forward.
    self.special_peft_forward_args = {"adapter_names"}

    self._is_prompt_learning = peft_config.is_prompt_learning
    self._peft_config = None
    self.base_model = CustomLoraModel(
        model, {adapter_name: peft_config}, adapter_name
    )

    self.set_additional_trainable_modules(peft_config, adapter_name)

    if getattr(model, "is_gradient_checkpointing", True):
      model = self._prepare_model_for_gradient_checkpointing(model)

    # the `pretraining_tp` is set for some models to simulate Tensor
    # Parallelism during inference to avoid numerical differences,
    # ✨ https://github.com/pytorch/pytorch/issues/76232 - to avoid any unexpected
    # behavior we disable that in this line.
    if hasattr(self.base_model, "config") and hasattr(
        self.base_model.config, "pretraining_tp"
    ):
      self.base_model.config.pretraining_tp = 1

## Train

In [ ]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=64,
    lora_dropout=0.01,
    bias="none",
    target_modules=[
        "q_proj",
        "o_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    task_type="CAUSAL_LM",
)

MODEL = reload_model()
MODEL = CustomPeftModel(MODEL, lora_config)
trainer = train(model=MODEL, lora_config=lora_config, num_epochs=50)
print(print_trainable_parameters(MODEL))


## Eval

In [ ]:
# @title Sample Eval
query_model(
    'Question: Janet’s ducks lay 16 eggs per day. She eats three for breakfast'
    ' every morning and bakes muffins for her friends every day with four. She'
    " sells the remainder at the farmers' market daily for $2 per fresh duck"
    " egg. How much in dollars does she make every day at the farmers' market?"
    ' \nAnswer: '
)

In [ ]:
# @title Bulk Evals Again
accuracy = run_bulk_evals(
    small_gsm8k_test,
    prompt_prefix='Question: ',
    prompt_suffix=' \nAnswer: ',
    answer_pattern='The answer is',
    verbose=True,
)
print('Accuracy: ', accuracy * 100, '%')

# Other PEFT Techniques


1.   Quantization [doc](https://huggingface.co/docs/peft/en/developer_guides/quantization)
2.   Quantization + LoRA [doc](https://huggingface.co/blog/4bit-transformers-bitsandbytes)
3.   Soft prompt tuning [doc](https://huggingface.co/docs/peft/en/conceptual_guides/prompting)



# ✨ Resources used for this tutorial and references



1. [Gemma Architecture](https://developers.googleblog.com/en/gemma-explained-overview-gemma-model-family-architectures/)  
2. [Huggingface LoRA](https://huggingface.co/docs/peft/main/en/conceptual_guides/lora)
3.   [Huggingface Finetuning](https://huggingface.co/docs/transformers/en/training)
4. [Finetuning Gemma for Math](https://medium.com/google-developer-experts/fine-tuning-gemma-2b-to-solve-math-problems-ac4921ed531e)
5. [Gemma PyTorch](https://github.com/google/gemma_pytorch)
6. [Gemma Finetuning](https://github.com/google-deepmind/gemma/blob/main/colabs/fine_tuning_tutorial.ipynb)
7. [LLM Optimizers](https://towardsdatascience.com/fine-tuning-llms-with-32-bit-8-bit-and-paged-adamw-optimizers-1034e3105634)
8. [LoRA Hypers](https://medium.com/@fartypantsham/what-rank-r-and-alpha-to-use-in-lora-in-llm-1b4f025fd133)
9. [Simple Gemma PEFT](https://huggingface.co/blog/gemma-peft)
10. [M2Lschool Previous Tutorial](https://github.com/M2Lschool/tutorials2024/blob/main/1_nlp/part_III_llm_finetuning/LoRA.ipynb)
